# Data Preparation

In [189]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import torch

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

from torch import nn

In [158]:
df = pd.read_csv('survey_results_public.csv')
df

,Unnamed: 0,Country,EdLevel,YearsCodePro,Employment,Salary
0,2,United Kingdom of Great Britain and Northern I...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Employed, full-time",40205.0
1,3,Israel,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",17,"Employed, full-time",215232.0
2,8,Netherlands,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",6,"Employed, full-time",49056.0
3,10,United Kingdom of Great Britain and Northern I...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",2,"Employed, full-time",60307.0
4,11,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,"Employed, full-time;Independent contractor, fr...",194400.0
...,...,...,...,...,...,...
37914,73116,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",16,"Employed, full-time",115000.0
37915,73118,Nigeria,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",3,"Employed, full-time",57720.0
37916,73119,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",1,"Employed, full-time",70000.0
37917,73120,Spain,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",2,"Employed, full-time",35192.0


In [159]:
# Drop kolom unamed
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,Country,EdLevel,YearsCodePro,Employment,Salary
0,United Kingdom of Great Britain and Northern I...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Employed, full-time",40205.0
1,Israel,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",17,"Employed, full-time",215232.0
2,Netherlands,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",6,"Employed, full-time",49056.0
3,United Kingdom of Great Britain and Northern I...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",2,"Employed, full-time",60307.0
4,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,"Employed, full-time;Independent contractor, fr...",194400.0


In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37919 entries, 0 to 37918
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country       37919 non-null  object 
 1   EdLevel       37919 non-null  object 
 2   YearsCodePro  37919 non-null  object 
 3   Employment    37919 non-null  object 
 4   Salary        37919 non-null  float64
dtypes: float64(1), object(4)
memory usage: 1.4+ MB


In [161]:
# Ambil atribut non-null pada salary
df = df[df['Salary'].notnull()]
df.shape

(37919, 5)

In [162]:
df = df.dropna()
df.isnull().sum()

Country         0
EdLevel         0
YearsCodePro    0
Employment      0
Salary          0
dtype: int64

## Preparation Employment Column

In [163]:
df['Employment'].value_counts()

Employed, full-time                                                                             31382
Independent contractor, freelancer, or self-employed                                             2739
Employed, full-time;Independent contractor, freelancer, or self-employed                         2633
Employed, part-time                                                                               632
Independent contractor, freelancer, or self-employed;Employed, part-time                          207
Employed, full-time;Employed, part-time                                                           151
Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time      118
I prefer not to say                                                                                27
Retired                                                                                            18
Employed, part-time;Retired                                                       

In [164]:
df = df[df['Employment'] != 'I prefer not to say']
df = df[~df['Employment'].str.contains('retired', case=False)]

df['Employment'].value_counts()

Employed, full-time                                                                             31382
Independent contractor, freelancer, or self-employed                                             2739
Employed, full-time;Independent contractor, freelancer, or self-employed                         2633
Employed, part-time                                                                               632
Independent contractor, freelancer, or self-employed;Employed, part-time                          207
Employed, full-time;Employed, part-time                                                           151
Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time      118
Name: Employment, dtype: int64

In [165]:
# Pisahkan ke baris baru dipisahkan dengan koma pada kolom Employment
df['Employment'] = df['Employment'].str.split(', ')
# Pisahkan ke baris baru
df = df.explode('Employment')
df['Employment'].value_counts()

Employed                            34916
full-time                           31382
freelancer                           5697
or self-employed                     5372
Independent contractor               2946
full-time;Independent contractor     2751
part-time                            1108
or self-employed;Employed             325
full-time;Employed                    151
Name: Employment, dtype: int64

In [166]:
# Pisahkan ke baris baru dipisahkan dengan titik koma pada kolom Employment
df['Employment'] = df['Employment'].str.split(';')
# Pisahkan ke baris baru
df = df.explode('Employment')
df['Employment'].value_counts()

Employed                  35392
full-time                 34284
Independent contractor     5697
freelancer                 5697
or self-employed           5697
part-time                  1108
Name: Employment, dtype: int64

In [167]:
# Menghilangkan kata or pada nilai or self-employed
df['Employment'] = df['Employment'].str.replace('or ', '')
df['Employment'].value_counts()

Employed                  35392
full-time                 34284
Independent contractor     5697
freelancer                 5697
self-employed              5697
part-time                  1108
Name: Employment, dtype: int64

In [168]:
# Drop kolom dengan nilai employment = employed
df = df[df['Employment'] != 'Employed']
df['Employment'].value_counts()

full-time                 34284
Independent contractor     5697
freelancer                 5697
self-employed              5697
part-time                  1108
Name: Employment, dtype: int64

## Preparation Country Column

In [169]:
df['Country'].value_counts()

United States of America                                10598
Germany                                                  3816
United Kingdom of Great Britain and Northern Ireland     3274
India                                                    2884
Canada                                                   2030
                                                        ...  
Suriname                                                    1
Haiti                                                       1
Fiji                                                        1
Cape Verde                                                  1
Seychelles                                                  1
Name: Country, Length: 159, dtype: int64

In [170]:
# Melakukan filter pada kolom Country jika nilai kurang dari cutoff
def convert_to_categories(categories, cutoff):
    categorical_map = {}
    for i in range(len(categories.value_counts())):
        if categories.value_counts().iloc[i] > cutoff:
            categorical_map[categories.value_counts().index[i]] = categories.value_counts().index[i]
        else:
            categorical_map[categories.value_counts().index[i]] = 'Other'
    return categories.map(categorical_map)

In [171]:
df['Country'] = convert_to_categories(df['Country'], 800)
df['Country'].value_counts()

Other                                                   18499
United States of America                                10598
Germany                                                  3816
United Kingdom of Great Britain and Northern Ireland     3274
India                                                    2884
Canada                                                   2030
Brazil                                                   1930
Poland                                                   1916
France                                                   1898
Spain                                                    1297
Netherlands                                              1247
Italy                                                    1110
Australia                                                1101
Sweden                                                    883
Name: Country, dtype: int64

## Preparation YearsCodePro Column

In [172]:
df['YearsCodePro'] = df['YearsCodePro'].replace({'Less than 1 year': 0, 'More than 50 years': 51}).astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52483 entries, 0 to 37918
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country       52483 non-null  object 
 1   EdLevel       52483 non-null  object 
 2   YearsCodePro  52483 non-null  int32  
 3   Employment    52483 non-null  object 
 4   Salary        52483 non-null  float64
dtypes: float64(1), int32(1), object(3)
memory usage: 2.2+ MB


In [173]:
df['YearsCodePro'].unique()

array([ 5, 17,  6,  2, 10,  4, 22, 20,  9, 14, 21, 15,  3, 25,  7,  8, 12,
       19,  1, 28, 24, 11, 23, 45,  0, 18, 40, 37, 46, 13, 32, 31, 39, 27,
       16, 26, 30, 34, 35, 29, 42, 38, 36, 43, 41, 44, 48, 33, 51, 50, 47,
       49])

## Preparation EdLevel Column

In [174]:
df['EdLevel'].value_counts()

Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                          24069
Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                                       13709
Some college/university study without earning a degree                                 6736
Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     2613
Associate degree (A.A., A.S., etc.)                                                    1800
Other doctoral degree (Ph.D., Ed.D., etc.)                                             1776
Professional degree (JD, MD, etc.)                                                      912
Something else                                                                          521
Primary/elementary school                                                               347
Name: EdLevel, dtype: int64

In [175]:
def education_converter(x):
    if 'Bachelor’s degree' in x:
        return 'Bachelor'
    elif 'Master’s degree' in x:
        return 'Master'
    elif 'Associate degree' in x:
        return 'Associate'
    elif 'Professional degree' in x:
        return 'Professional'
    elif 'Other doctoral degree' in x:
        return 'Doctoral'
    elif 'Secondary school' in x:
        return 'Secondary school'
    elif 'Primary' in x:
        return 'Primary school'
    else:
        return 'Other degree'
    
df['EdLevel'] = df['EdLevel'].apply(education_converter)
df['EdLevel'].value_counts()

Bachelor            24069
Master              13709
Other degree         7257
Secondary school     2613
Associate            1800
Doctoral             1776
Professional          912
Primary school        347
Name: EdLevel, dtype: int64

## Summary

In [176]:
df['Country'].value_counts()

Other                                                   18499
United States of America                                10598
Germany                                                  3816
United Kingdom of Great Britain and Northern Ireland     3274
India                                                    2884
Canada                                                   2030
Brazil                                                   1930
Poland                                                   1916
France                                                   1898
Spain                                                    1297
Netherlands                                              1247
Italy                                                    1110
Australia                                                1101
Sweden                                                    883
Name: Country, dtype: int64

In [177]:
df['YearsCodePro'].value_counts()

5     4207
3     4146
4     3796
10    3503
2     3489
6     3335
7     2962
8     2915
1     2282
15    1982
12    1860
9     1737
0     1559
11    1485
20    1469
14    1187
13    1049
16     976
22     951
25     875
17     874
18     846
30     544
23     494
24     490
21     456
19     413
26     347
27     287
28     257
35     248
32     229
40     152
31     130
29     125
37     119
36     111
34     111
33     103
38      92
42      56
39      48
41      35
43      29
45      27
44      24
51      17
48      15
46      11
49      11
50      10
47       7
Name: YearsCodePro, dtype: int64

In [178]:
df['EdLevel'].value_counts()

Bachelor            24069
Master              13709
Other degree         7257
Secondary school     2613
Associate            1800
Doctoral             1776
Professional          912
Primary school        347
Name: EdLevel, dtype: int64

In [179]:
df['Employment'].value_counts()

full-time                 34284
Independent contractor     5697
freelancer                 5697
self-employed              5697
part-time                  1108
Name: Employment, dtype: int64

In [180]:
df.head()

,Country,EdLevel,YearsCodePro,Employment,Salary
0,United Kingdom of Great Britain and Northern I...,Master,5,full-time,40205.0
1,Other,Bachelor,17,full-time,215232.0
2,Netherlands,Master,6,full-time,49056.0
3,United Kingdom of Great Britain and Northern I...,Bachelor,2,full-time,60307.0
4,United States of America,Bachelor,10,full-time,194400.0


# Preprocessing Data

In [181]:
ed_encoder = LabelEncoder()
df['EdLevel'] = ed_encoder.fit_transform(np.ravel(df[['EdLevel']].values))
print(ed_encoder.classes_)
print(df['EdLevel'].unique())

['Associate' 'Bachelor' 'Doctoral' 'Master' 'Other degree'
 'Primary school' 'Professional' 'Secondary school']
[3 1 7 4 0 6 2 5]


In [182]:
country_encoder = LabelEncoder()
df['Country'] = country_encoder.fit_transform(np.ravel(df[['Country']].values))
print(country_encoder.classes_)
print(df['Country'].unique())

['Australia' 'Brazil' 'Canada' 'France' 'Germany' 'India' 'Italy'
 'Netherlands' 'Other' 'Poland' 'Spain' 'Sweden'
 'United Kingdom of Great Britain and Northern Ireland'
 'United States of America']
[12  8  7 13  6  2  4  9  3  1 11 10  5  0]


In [183]:
employment_encoder = LabelEncoder()
df['Employment'] = employment_encoder.fit_transform(np.ravel(df[['Employment']].values))
print(employment_encoder.classes_)
print(df['Employment'].unique())

['Independent contractor' 'freelancer' 'full-time' 'part-time'
 'self-employed']
[2 0 1 4 3]


# Convert Data to New Data Cleaning

In [184]:
df.to_csv('survey_results_public_clean.csv', index=False)

df = pd.read_csv('survey_results_public_clean.csv')
df.head()

,Country,EdLevel,YearsCodePro,Employment,Salary
0,12,3,5,2,40205.0
1,8,1,17,2,215232.0
2,7,3,6,2,49056.0
3,12,1,2,2,60307.0
4,13,1,10,2,194400.0


# Data Modelling

In [186]:
X = df.drop('Salary', axis=1)
y = df['Salary']